In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.8


In [2]:
from classifier.create_ws import AZHelper
ws = AZHelper.load_ws()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: /home/aaditya/PycharmProjects/Codefundopp/classifier/aml_config/config.json


Workspace name: pyt1
Azure region: eastus2
Subscription id: af7772d6-3275-4058-8d9e-7640742004e3
Resource group: Codefundopp


In [4]:
compute_target = AZHelper.load_cp(ws)

creating a new compute target...


Creating


Succeeded

AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-01-21T09:43:50.888000+00:00', 'creationTime': '2019-01-21T09:40:14.475546+00:00', 'currentNodeCount': 0, 'errors': None, 'modifiedTime': '2019-01-21T09:43:56.853257+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'targetNodeCount': 0, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [5]:
import os

project_folder = './pytorch-distr-hvd'
os.makedirs(project_folder, exist_ok=True)

In [ ]:
import shutil

shutil.copy('classifier/train_on_azure.py', project_folder)
shutil.copy('classifier/dataset.py', project_folder)
shutil.copy('classifier/xception.py', project_folder)